In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def V(m,x,x_eq=-3,omega=0.001):
    y=m/2*omega**2*(x-x_eq)**2
    return y
def Gauss(x,beta=1,x_eq=0):
    y=np.exp(-beta*(x-x_eq)**2)
    return y
def k(N_G,dk):
    l=np.zeros(N_G)
    for i in range(0,N_G):
        if i<=N_G/2+1:
            l[i]=i*dk
        else:
            l[i]=-(N_G-i)*dk
    return l
def Vprop(V,dt,itp=False):
    if itp:
        im=1
    else:
        im=1.0j
    l=np.exp(-im*V*dt)
    return l
def Tprop(k,m,dt,itp=False):
    if itp:
        im=1
    else:
        im=1.0j
    l=np.exp(-im/(2*m)*k**2*dt)
    return l
def H(m,k,V):
    l=k**2/(2*m)+V
    return l
def Norm(f,dx):
    l=np.real(np.sum(np.conjugate(f)*f)*dx)
    return l

In [ ]:
N_G=1025
Grid_min=-10
Grid_max=10
om=0.002
dx=(Grid_max-Grid_min)/(N_G-1)
dk=2*np.pi/(N_G*dx)

n_t=100000
t_initial=-5000
t_final=5000
dt=(t_final-t_initial)/(n_t-1)

m=1836
n_s=2

W_12=0.01

omega_f=0.098
beta_t=0.0000001
tau_0=0
E_0=0.01

In [ ]:
x_grid=np.linspace(Grid_min,Grid_max,num=N_G)
k_grid=k(N_G,dk)
t_grid=np.linspace(t_initial, t_final, n_t)

E_grid=E_0*np.exp(-beta_t*(t_grid-tau_0)**2)
Welle_grid=E_grid*np.cos(omega_f*(t_grid-tau_0))

T_grid=np.zeros((n_s,n_s,N_G),dtype=np.complex128)
T_grid[0,0,:]=Tprop(k(N_G,dk),m,dt)
T_grid[1,1,:]=Tprop(k(N_G,dk),m,dt)

pot1=V(m,x_grid, x_eq=-3, omega=om)
pot2=V(m,x_grid, x_eq=3, omega=om)

V_grid=np.zeros((n_s,n_s,N_G),dtype=np.complex128)
V_grid[0,0,:]=Vprop(pot1,dt/2)
V_grid[1,1,:]=Vprop(pot2,dt/2)

W_grid=np.zeros((n_s,n_s,n_t),dtype=np.complex128)
W_grid[0,0,:]=np.cos(Welle_grid*dt/4)
W_grid[0,1,:]=-1.0j*np.sin(Welle_grid*dt/4)
W_grid[1,0,:]=-1.0j*np.sin(Welle_grid*dt/4)
W_grid[1,1,:]=np.cos(Welle_grid*dt/4)

In [ ]:
wf=np.zeros((n_s,N_G))
wf[0,:]=0. #Gauss(x_grid,beta=1,x_eq=-8)
wf[1,:]=Gauss(x_grid,beta=0.3,x_eq=3)
wf/=np.sqrt(Norm(wf,dx))
print(Norm(wf[0],dx),Norm(wf[1],dx))
#plt.plot(wf[0,:])
#plt.plot(wf[1,:])
#plt.show()
plt.plot(x_grid, pot1)
plt.plot(x_grid, pot2)
plt.plot(x_grid,wf[1,:])
plt.show()
print(x_grid[len(x_grid)//2])
print(pot1[len(pot1)//2]*27.2118)
print(pot2[len(pot2)//2]*27.2118)

plt.plot(t_grid,Welle_grid)
plt.show()
print(pot2[400])

In [ ]:
wf_1=Gauss(x_grid,beta=1,x_eq=-8)
wf_1/=np.sqrt(Norm(Gauss(x_grid),dx))
iT_grid=Tprop(k(N_G,dk),m,dt,itp=True)
iV_grid=Vprop(V(m,x_grid,x_eq=-3,omega=om),dt/2,itp=True)
E_0=100
E_1=1
print(wf_1)
while np.absolute(E_1-E_0)>=10**(-12):
    E_0=E_1
    wf_1=iV_grid*wf_1
    wf_1=np.fft.fft(wf_1)
    wf_1=iT_grid*wf_1
    wf_1=np.fft.ifft(wf_1)
    wf_1=iV_grid*wf_1
    n=Norm(wf_1,dx)
    E_1=-1/(2*dt)*np.log(n)
    wf_1=(1.0+0j)*np.real(wf_1/np.sqrt(n))
    #print(E_1)
print(E_1,wf_1)
plt.plot(x_grid,wf_1)
plt.show()

In [ ]:
wf_store=np.zeros((n_s,N_G,n_t),dtype=np.complex128)
op="ij...,j...->i..."
n1_store=np.zeros(len(t_grid))
n2_store=np.zeros(len(t_grid))

wf=np.zeros((n_s,N_G),dtype=np.complex128)
wf[0,:]=wf_1 #0. #Gauss(x_grid,beta=1,x_eq=-8)
wf[1,:]=0 #Gauss(x_grid,beta=0.3,x_eq=3)
wf/=np.sqrt(Norm(wf,dx))
print(wf[0,:])
print(Norm(wf[0],dx),Norm(wf[1],dx))

for i, t in enumerate(t_grid):
    #print(wf.shape)
    wf=np.einsum(op,W_grid[:,:,i],wf)
    wf=np.einsum(op,V_grid,wf)
    wf=np.einsum(op,W_grid[:,:,i],wf)
    wf=np.fft.fft(wf)
    wf=np.einsum(op,T_grid,wf)
    wf=np.fft.ifft(wf)
    wf=np.einsum(op,W_grid[:,:,i],wf)
    wf=np.einsum(op,V_grid,wf)
    wf=np.einsum(op,W_grid[:,:,i],wf)
    wf_store[:,:,i]=wf
    #print(Norm(wf,dx))
    n1_store[i]=np.real(Norm(wf[0,:],dx))
    n2_store[i]=np.real(Norm(wf[1,:],dx))
    #print(n1_store,n2_store)

In [ ]:
X,Y=np.meshgrid(t_grid[::10],x_grid)
plt.contourf(X,Y,np.real(np.abs(wf_store[0,:,::10])**2),levels=50,cmap="viridis_r")
plt.show()
plt.contourf(X,Y,np.real(np.abs(wf_store[1,:,::10])**2),levels=50,cmap="viridis_r")
plt.show()
plt.plot(t_grid,n1_store)
plt.plot(t_grid,n2_store)
plt.plot(t_grid,n1_store+n2_store)
plt.show()